In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [ ]:
# CREDS = 'is3107-418808-5bca2b8d29d0.json'
CREDS = 'is3107-418011-f63573e5e1f3.json'
credentials = service_account.Credentials.from_service_account_file(CREDS)
client = bigquery.Client(credentials=credentials)
job_config = bigquery.QueryJobConfig()

In [ ]:
dataset_id='is3107-418011.is3107'
print(list(client.list_datasets(credentials.project_id)))
tables = client.list_tables(dataset_id)  
for table in tables:
    print(table.table_id)

In [5]:
df = pd.read_csv('data_sets/data_with_location_info.csv')
df.head()

/var/folders/l5/0p3sf6hj6sqd3k5fbpl4prrw0000gn/T/ipykernel_48927/313019750.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_sets/data_with_location_info.csv')


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,flat_type_model,school_type,bus_stop_proximity,mrt_proximity,pri_sch_proximity,sec_sch_proximity,inflation,normalized_resale_price,latitude,longitude
0,1998-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,10 TO 12,73.0,NEW GENERATION,1976,170500.0,...,3 ROOM NEW GENERATION,good,3-5 minutes,more than 15 minutes,5-10 minutes,3-5 minutes,1.324981,225909.328483,1.366197,103.841505
1,1998-05,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,NEW GENERATION,1976,170000.0,...,3 ROOM NEW GENERATION,good,3-5 minutes,more than 15 minutes,5-10 minutes,3-5 minutes,1.438463,244538.767743,1.366197,103.841505
2,1998-07,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,01 TO 03,73.0,NEW GENERATION,1976,157000.0,...,3 ROOM NEW GENERATION,good,3-5 minutes,more than 15 minutes,5-10 minutes,3-5 minutes,1.652590,259456.595072,1.366197,103.841505
3,1998-07,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,01 TO 03,73.0,NEW GENERATION,1976,150000.0,...,3 ROOM NEW GENERATION,good,3-5 minutes,more than 15 minutes,5-10 minutes,3-5 minutes,1.652590,247888.466630,1.366197,103.841505
4,1998-07,ANG MO KIO,4 ROOM,216,ANG MO KIO AVE 1,10 TO 12,88.0,NEW GENERATION,1976,263000.0,...,4 ROOM NEW GENERATION,good,3-5 minutes,more than 15 minutes,5-10 minutes,3-5 minutes,1.652590,434631.111491,1.366197,103.841505


In [9]:
mixed_type_columns = df.select_dtypes(include='object').columns

if mixed_type_columns.empty:
    print("No columns with mixed data types.")
else:
    print("Columns with mixed data types:")
    print(mixed_type_columns)
df[mixed_type_columns] = df[mixed_type_columns].astype(str)

Columns with mixed data types:
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'flat_model', 'address', 'planning_area', 'mrt_name', 'bus_stop_name',
       'pri_sch_name', 'sec_sch_name', 'flat_type_model', 'school_type',
       'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity',
       'sec_sch_proximity'],
      dtype='object')


In [ ]:
# months = ['January', 'February', 'March', 'April', 'May', 'June', 
#           'July', 'August', 'September', 'October', 'November', 'December']
# 
# df = pd.DataFrame({"month": months})
# df

In [10]:
project_id = "is3107-418011"

# Dataset ID and table name
dataset_id = "is3107"  # You have nested dataset, so specify only the innermost dataset
table_id = "resale_price"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 6626.07it/s]


In [2]:
def query_table_from_bq(table_name):
    CREDS = 'is3107-418011-f63573e5e1f3.json'
    credentials = service_account.Credentials.from_service_account_file(CREDS)
    client = bigquery.Client(credentials=credentials)
    job_config = bigquery.QueryJobConfig()

    # Set the destination table
    table = client.dataset(dataset_id='is3107-418011.is3107').table(table_id=f"is3107-418011.is3107.{table_name}")
    job_config.destination = table
    query = f"SELECT * FROM `is3107-418011.is3107.{table_name}`"
    return client.query(query).to_dataframe()

In [3]:
query_table_from_bq("testing2")

,month
0,January
1,February
2,March
3,April
4,May
5,June
6,July
7,August
8,September
9,October
